# **基于MindSpore下的人脸识别分类**

## 1 模型介绍
#### 1.1 模型简介

在大数据时代，图像数据激增，计算机视觉技术的重要性日益凸显。深度学习作为一种端到端的学习范式，能够自动从大量数据中学习到丰富的特征表示，在视觉识别任务中展现出了强大的性能。从早期简单的模板匹配、手工设计特征的方法，到如今基于深度神经网络的端到端学习框架，视觉识别技术经历了巨大的变革。

在深度学习兴起之前，视觉识别主要依赖手工提取特征，如SIFT、HOG等，然后利用传统机器学习算法如SVM进行分类。这种方法在计算资源有限的时代取得了一定成果，但存在明显缺陷：特征提取依赖专家知识，泛化能力差，且难以应对图像的复杂变化如光照、姿态、遮挡等。随着图像数据量爆炸式增长，对更高效、更智能的识别方法的需求愈发迫切。

2012 年，Hinton 和他的学生 Alex 在论文《ImageNet Classification with Deep Convolutional Neural Networks》中提出的 AlexNet，成为了视觉识别领域的重要转折点。它在当年的 ILSVRC 竞赛中一举夺冠，top-5 识别错误率比第二名低近 10%，引发业界轰动。AlexNet 的成功并非偶然，而是多种创新技术的结合：它采用多 GPU 训练，突破计算资源限制；引入 ReLU 激活函数，加速训练且缓解梯度消失；应用 Dropout 技术，有效防止过拟合；使用局部响应归一化（LRN），增强特征表达能力。这些技术使 AlexNet 在复杂图像分类任务中展现出前所未有的性能，开启了深度学习在视觉识别领域的统治地位。

#### 1.2 模型详解

<img src="./image/Alexnet.png">

AlexNet 有 8 层权重层，包括 5 层卷积层和 3 层全连接层（FC 层），并引入了一些重要的创新，包括激活函数、Dropout 正则化和重叠池化。它通过增加网络的深度和宽度，结合 GPU 加速，极大提升了 CNN 的能力。

1. **卷积+池化层（前五层）**
AlexNet共有五个卷积层，每个卷积层都包含卷积核、偏置项、ReLU激活函数和局部响应归一化（LRN）模块。

卷积层C1：使用96个核对224 × 224 × 3的输入图像进行滤波，卷积核大小为11 × 11 × 3，步长为4。将一对55×55×48的特征图分别放入ReLU激活函数，生成激活图。激活后的图像进行最大池化，size为3×3，stride为2，池化后的特征图size为27×27×48（一对）。池化后进行LRN处理。

卷积层C2：使用卷积层C1的输出（响应归一化和池化）作为输入，并使用256个卷积核进行滤波，核大小为5 × 5 × 48。

卷积层C3：有384个核，核大小为3 × 3 × 256，与卷积层C2的输出（归一化的，池化的）相连。

卷积层C4：有384个核，核大小为3 × 3 × 192。

卷积层C5：有256个核，核大小为3 × 3 × 192。卷积层C5与C3、C4层相比多了个池化，池化核size同样为3×3，stride为2。

其中，卷积层C3、C4、C5互相连接，中间没有接入池化层或归一化层。

2. **全连接层（后三层）**
全连接层F6：因为是全连接层，卷积核size为6×6×256，4096个卷积核生成4096个特征图，尺寸为1×1。然后放入ReLU函数、Dropout处理。值得注意的是AlexNet使用了Dropout层，以减少过拟合现象的发生。

全连接层F7：同F6层。

全连接层F8：最后一层全连接层的输出是1000维softmax的输入，softmax会产生1000个类别预测的值。

#### 1.3 AlexNet主要创新点

1. **ReLU激活函数的引入**

采用修正线性单元(ReLU)的深度卷积神经网络训练时间比等价的tanh单元要快几倍。而时间开销是进行模型训练过程中很重要的考量因素之一。同时，ReLU有效防止了过拟合现象的出现。由于ReLU激活函数的高效性与实用性，使得它在深度学习框架中占有重要地位。

2. **层叠池化操作**

以往池化的大小PoolingSize与步长stride一般是相等的，例如：图像大小为256*256，PoolingSize=2×2，stride=2，这样可以使图像或是FeatureMap大小缩小一倍变为128，此时池化过程没有发生层叠。但是AlexNet采用了层叠池化操作，即PoolingSize > stride。这种操作非常像卷积操作，可以使相邻像素间产生信息交互和保留必要的联系。论文中也证明，此操作可以有效防止过拟合的发生。

3. **Dropout操作**

Dropout操作会将概率小于0.5的每个隐层神经元的输出设为0，即去掉了一些神经节点，达到防止过拟合。那些“失活的”神经元不再进行前向传播并且不参与反向传播。这个技术减少了复杂的神经元之间的相互影响。在论文中，也验证了此方法的有效性。


#### **1.4 模型特点**

##### a) 非线性激活函数 ReLU:

标准的L-P神经元的输出一般使用$f(x) = tanh(x)$或$f(x) = (1+e^{-x})^{-1}$作为激活函数，而在训练阶段梯度衰减快慢方面，AlexNet所使用的ReLU函数，即$f(x) = max(0, x)$,比上述两个非线性饱和函数要快许多。效果详见下图：


##### b) 多GPU训练:

在当时，单个GPU的内存限制了网络的训练规模，而采用多GPU训练可解决上述问题，提高AlexNet的训练速度。

但随着技术的发展，在绝大多数情况下，单个GPU的内存已经足以支持整个模型的训练，而多GPU训练也成为了AlexNet这个项目中最复杂的技术细节。此外，作者在此处提出的双GPU训练的可复现性差，所以在未来的几年内并没有被重视。

然而近些年随着体量更大的模型的出现，GPU瓶颈再次产生，这使得模型并行、分布训练（多GPU训练）又成为了大家解决GPU瓶颈的重要技术。

##### c) 局部响应归一化：

采用局部响应归一化(local response normalization，LRN)可将数据分布调整到合理的范围内，便于计算处理，从而提高泛化能力，避免过饱和。

具体公式如下：

$${b}_{x,y}^{i}={a}_{x,y}^{i}/({{k}+{α}{\sum_{j=max(0,i-n/2)}^{min(N-1,i+n/2)}}({a}_{x,y}^{i})^2})^{β}$$

但从现在的角度来看，LRN技术的作用并不大，而且在之后的训练中也并未得到广泛的使用。

##### d) 重叠池化层：

一般来说两个pooling是不重叠的，但是这里采用了一种对传统的pooling改进的方式，不仅可以提升预测精度，同时一定程度上可以减缓过拟合。


#### **1.5 降低过拟合**

##### a) 数据增强：

减少图像数据过度拟合的最简单也是最常见的方法是使用保留标签的变换人为地放大数据集。这里用了两种方式来实现数据增强：

i）随机区域提取：该方法在256×256的图片上随机提取一个224×224的区域用以训练，这使得数据集的大小变为原来的2048倍。

ii）RGB通道改变：该方法采用PCA的方式对RGB图像的channel进行了一些改变，使图像发生了一些变化，从而扩大了数据集。

##### b) dropout:

Dropout通过设置好的概率（50%），随机将某个隐藏层神经元的输出设置为0，因此这个神经元将不参与前向传播和反向传播，在下一次迭代中会根据概率重新将某个神经元的输出置0。简单来说就是在全连接层中去掉了一些神经节点，达到了防止过拟合的目的。但dropout的使用也让AlexNet的训练速度慢了一倍。Dropout相当于一个L2的正则化，只不过用了这种方式实现了L2正则化的功能。

#### **1.6 模型参数**

超参数：使用随机梯度下降法（SGD）训练模型，batch size = 128，momentum = 0.9，权重衰减 = 0.0005，学习率=0.01.

初始化：首先用均值为0，方差为0.01的高斯随机变量初始化权重参数，再将第2、4、5个卷积层和全连接起来的隐藏层的神经元偏置初始化为常数1，最后将剩余层的神经元偏置初始化为常数0.

学习率：在所有层上均使用0.01的学习率，验证集的错误率不再随当前学习率提高时，将学习率除以10.

### **2 案例实现**

#### **2.1 数据集**

本文采用CASIA-WebFace中的人脸识别数据集作为训练集，CASIA-WebFace是一款免费开源的用于人脸识别的数据集，它是从IMDb网站上进行数据爬取，网站中具有非常多人脸特征，并且数据集里不会有重复特征，目前共有一万五千个名人数据集和四十九万张照片。

采用LFW数据集作为训练集， LFW (Labled Faces in the Wild)人脸数据集：是目前人脸识别的常用测试集，其中提供的人脸图片均来源于生活中的自然场景，因此识别难度会增大，尤其由于多姿态、光照、表情、年龄、遮挡等因素影响导致即使同一人的照片差别也很大。并且有些照片中可能不止一个人脸出现，对这些多人脸图像仅选择中心坐标的人脸作为目标，其他区域的视为背景干扰。LFW数据集共有13233张人脸图像，每张图像均给出对应的人名，共有5749人，且绝大部分人仅有一张图片。每张图片的尺寸为250X250，绝大部分为彩色图像，但也存在少许黑白人脸图片。
下载地址：<a>http://vis-www.cs.umass.edu/lfw/lfw.tgz</a>


#### **2.2 环境准备与数据读取**

本案例基于MindSpore-CPU版本实现，在CPU上完成模型训练。

环境需要mindspore和mindvision。

In [1]:
import mindspore
import mindspore.nn as nn

from mindspore import ops
from mindvision import dataset
from mindspore.dataset import vision

在CASIA-WebFace中它的数据集里还会对爬下来的照片进行裁剪，忽略掉哪些杂项背景，只保留人脸区域，同时每张照片会保留两份，来加强数据集的数据重复度以便增强可信度。并且它的尺寸都被统一缩减为96x96，在图像识别方面将图像缩减到96x96对图像的像素损失最低同时尺寸大小也会变小，并且数据量也会减少，这对后续的卷积与池化会更快。

数据集图像示例：
<img src="./image/图片.png">

本案例通过CASIA-WebFace下载训练数据和通过LFW下载测试数据，下载后的数据集会保存在设定好的路径。

In [2]:
# 从dataset下载训练数据
training_data = dataset.Cifar10(
    path="./datatests",
    split="train",
    download=True,
)

# 从dataset下载测试数据
test_data = dataset.Cifar10(
    path="./lfw",
    split="test",
    download=True,
)

# 数据下载完成后，获取数据集对象
train_dataset = training_data.dataset
test_dataset = test_data.dataset

# 查看数据集中包含的数据列名，用于dataset的预处理
train_dataset.column_names

['image', 'label']

#### 2.3 **规范数据集格式**

为了确保数据的一致性和可用性，需要对数据集进行格式规范处理。

In [3]:
import os
import shutil
import re

def reorganize_dataset(source_dir, target_dir):
    os.makedirs(target_dir, exist_ok=True)
    
    # 获取并排序子文件夹
    subfolders = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    try:
        subfolders.sort(key=lambda x: int(x))  # 按数字排序
    except ValueError:
        subfolders = sorted(subfolders)  # 备用字母排序
    
    for folder_idx, folder in enumerate(subfolders):
        old_folder = os.path.join(source_dir, folder)
        new_folder = os.path.join(target_dir, f"people{folder_idx}")
        os.makedirs(new_folder, exist_ok=True)
        
        # 获取并排序图片文件
        files = [f for f in os.listdir(old_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        # 提取数字并排序
        files.sort(key=lambda f: int(re.search(r'(\d+)', f).group()))
        
        # 复制并重命名
        for img_idx, filename in enumerate(files, 1):
            old_path = os.path.join(old_folder, filename)
            new_name = f"{img_idx:03d}{os.path.splitext(filename)[1]}"  # 保持原扩展名
            new_path = os.path.join(new_folder, new_name)
            
            shutil.copy2(old_path, new_path)
        
        print(f"转换完成: {folder} → people{folder_idx} ({len(files)}张图片)")

if __name__ == "__main__":
    source = "lfw"
    target = "./datatests"
    reorganize_dataset(source, target)

转换完成: AJ_Cook → people0 (1张图片)
转换完成: AJ_Lamas → people1 (1张图片)
转换完成: Aaron_Eckhart → people2 (1张图片)
转换完成: Aaron_Guiel → people3 (1张图片)
转换完成: Aaron_Patterson → people4 (1张图片)
转换完成: Aaron_Peirsol → people5 (4张图片)
转换完成: Aaron_Pena → people6 (1张图片)
转换完成: Aaron_Sorkin → people7 (2张图片)
转换完成: Aaron_Tippin → people8 (1张图片)
转换完成: Abba_Eban → people9 (1张图片)
转换完成: Abbas_Kiarostami → people10 (1张图片)
转换完成: Abdel_Aziz_Al-Hakim → people11 (1张图片)
转换完成: Abdel_Madi_Shabneh → people12 (1张图片)
转换完成: Abdel_Nasser_Assidi → people13 (2张图片)
转换完成: Abdoulaye_Wade → people14 (4张图片)
转换完成: Abdul_Majeed_Shobokshi → people15 (1张图片)
转换完成: Abdul_Rahman → people16 (1张图片)
转换完成: Abdulaziz_Kamilov → people17 (1张图片)
转换完成: Abdullah → people18 (4张图片)
转换完成: Abdullah_Ahmad_Badawi → people19 (1张图片)
转换完成: Abdullah_Gul → people20 (19张图片)
转换完成: Abdullah_Nasseef → people21 (1张图片)
转换完成: Abdullah_al-Attiyah → people22 (3张图片)
转换完成: Abdullatif_Sener → people23 (2张图片)
转换完成: Abel_Aguilar → people24 (1张图片)
转换完成: Abel_Pacheco → people25 (4张

通过上述代码，可以将原始数据集的文件夹和文件进行规范化处理，确保数据集的格式统一，便于后续的模型训练和评估。处理后的数据集结构如下：

```
    datatests/
        people0/
            001.jpg
            002.jpg
            ...
        people1/
            001.jpg
            002.jpg
            ...
        ...
```

这种规范化的数据格式有助于提高数据加载和预处理的效率，为模型训练提供高质量的数据支持。

#### **2.4 数据集处理**

我们对数据集进行转换，统一数据格式，并拆分成多个batch，通过创建数据集迭代器并打印部分数据的形状和类型，确保图像和标签的格式符合模型的输入要求。

In [4]:

# 这里我们设置batch大小，并定义图像所需要做的数据变换，包括‘Resize’、‘Rescale’、‘HWC2CHW’。
batch_size = 128

transforms = [
    vision.Resize((224, 224)),
    vision.Rescale(1.0 / 255.0, 0),
    vision.HWC2CHW()
]

# 创建训练集和测试集，并拆分成多个batch
train_dataset = train_dataset.map(transforms, 'image').batch(batch_size)
test_dataset = test_dataset.map(transforms, 'image').batch(batch_size)

for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (128, 3, 224, 224) Float32
Shape of label: (128,) Int32


#### **2.5 模型构建**

在本案例中，我们基于MindSpore框架对AlexNet模型进行了重新构建和优化。模型结构在遵循原论文《ImageNet Classification with Deep Convolutional Neural Networks》的基础上，针对单GPU训练环境对通道数进行了适当调整。具体来说，除第一个卷积层保持原通道数外，第二到第四个卷积层的通道数进行了合并，以适应计算资源的限制。

在模型定义方面，我们沿用了MindSpore框架的标准流程。首先，定义了conv函数和fc_with_initialize函数，用于自定义nn.Conv2d卷积层和nn.Dense全连接层的初始化。然后，在AlexNet类中重写了__init__和construct方法。在__init__方法中，我们依次定义了模型的各个层，包括五个卷积层、ReLU激活函数、最大池化层、Flatten层、三个全连接层以及Dropout层。其中，新增了两个卷积层（conv6和conv7）和一个全连接层（fc3），以进一步提升模型的特征提取和分类能力。

在construct方法中，我们构建了模型的前向传播过程。输入数据依次经过卷积层、激活函数、池化层等操作，逐步提取高级特征；然后通过Flatten层将特征图展平，输入到全连接层进行分类预测。新增的卷积层和全连接层融入到前向传播流程中，增强了模型的表达能力和学习能力。


In [5]:
import mindspore
import mindspore.nn as nn
from mindspore import ops
from mindspore import context
import mindspore.dataset as ds
from mindspore.train import Model, LossMonitor, TimeMonitor
from mindspore.train.callback import Callback
from mindspore.nn.metrics import Accuracy

# context.set_context(mode=context.GRAPH_MODE, device_target="GPU")  ## GPU调用

# 定义卷积层
def conv(in_channels, out_channels, kernel_size, stride=1, padding=0, pad_mode="pad", has_bias=True):
    """返回一个2D卷积模块"""
    return nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding,
                     has_bias=has_bias, pad_mode=pad_mode)

# 定义全连接层
def fc_with_initialize(input_channels, out_channels, has_bias=True):
    """获得一个全连接层"""
    return nn.Dense(input_channels, out_channels, has_bias=has_bias)

# 定义AlexNet模型
class AlexNet(nn.Cell):
    def __init__(self, num_classes=10, channel=3, dropout_ratio=0.5):
        """AlexNet模型
        Args:
            num_classes: 类别数
            channel    : 输入图像的通道数
        """
        super(AlexNet, self).__init__()
        self.channel = channel
        self.num_classes = num_classes
        self.dropout_ratio = dropout_ratio

        self.conv1 = conv(channel, 96, 11, stride=4, pad_mode='pad', padding=1, has_bias=True)
        self.conv2 = conv(96, 256, 5, pad_mode='pad', padding=2, has_bias=True)
        self.conv3 = conv(256, 384, 3, pad_mode='pad', padding=1, has_bias=True)
        self.conv4 = conv(384, 384, 3, pad_mode='pad', padding=1, has_bias=True)
        self.conv5 = conv(384, 256, 3, pad_mode='pad', padding=1, has_bias=True)
        self.conv6 = conv(256, 256, 3, pad_mode='pad', padding=1, has_bias=True)  # 新增卷积层
        self.conv7 = conv(256, 256, 3, pad_mode='pad', padding=1, has_bias=True)  # 新增卷积层

        self.relu = nn.ReLU()
        self.max_pool2d = nn.MaxPool2d(kernel_size=3, stride=2)
        self.flatten = nn.Flatten()

        self.fc1 = fc_with_initialize(6400, 4096)
        self.fc2 = fc_with_initialize(4096, 4096)
        self.fc3 = fc_with_initialize(4096, 4096)  # 新增全连接层
        self.fc4 = fc_with_initialize(4096, num_classes)

        self.dropout = nn.Dropout(dropout_ratio)

    def construct(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv6(x)  # 新增卷积层
        x = self.relu(x)
        x = self.conv7(x)  # 新增卷积层
        x = self.relu(x)
        x = self.max_pool2d(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc3(x)  # 新增全连接层
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc4(x)
        return x


#### **2.4 训练和测试函数的定义**

在模型训练中，一个完整的训练过程（step）需要实现以下三步：

1. 正向计算：模型预测结果（logits），并与正确标签（label）求预测损失（loss）。

2. 反向传播：利用自动微分机制，自动求模型参数（parameters）对于loss的梯度（gradients）。

3. 参数优化：将梯度更新到参数上。

MindSpore使用函数式自动微分机制，因此针对上述步骤需要实现：

1. 正向计算函数定义：定义一个函数forward_fn，该函数接受数据和标签作为输入，返回模型的预测结果和损失值。

2. 通过函数变换获得梯度计算函数：使用ops.value_and_grad函数，将forward_fn转换为一个能够同时计算损失和梯度的函数grad_fn。

3. 训练函数定义：定义一个训练步骤函数train_step，在该函数中执行正向计算、反向传播和参数优化。


In [6]:
# 定义训练函数
def model_train(model, dataset, loss_fn, optimizer):
    """训练模型
    Args:
        model    : 模型
        dataset  : 数据集
        loss_fn  : 损失函数
        optimizer: 优化器
    """
    # 前向传播函数
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # 获得梯度函数
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # 进行每一步训练
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()

    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)
        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")

##### **测试模型函数**

**1. 评估模型性能：**

**准确率（Accuracy）：** 计算模型在测试数据集上预测正确的样本比例，反映模型的分类能力。

**平均损失（Avg loss）：** 计算模型在测试数据集上的平均损失值，反映模型预测结果与真实标签之间的差异。

**2. 模型验证：**

通过在测试数据集上评估模型，验证模型的泛化能力，即模型在未见过的数据上的表现。

帮助判断模型是否存在过拟合或欠拟合问题。

In [7]:
# 定义测试函数
def model_test(model, dataset, loss_fn):
    """测试模型
    Args:
        model  : 模型
        dataset: 数据集
        loss_fn: 损失函数
    """
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0

    # 计算预测正确的样本个数
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    
    print(f"Test: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### **2.5 模型训练及评估**

在模型训练过程中，我们通过多次迭代数据集来优化模型参数，一次完整的迭代称为一个epoch。每个epoch包括训练和评估两个阶段：在训练阶段，模型在训练数据集上进行学习，更新参数以最小化损失函数；在评估阶段，模型在测试数据集上进行预测，评估其泛化能力和性能。训练过程中，我们会观察损失值（loss）和预测准确率（Accuracy）的变化趋势，理想情况下，随着训练的进行，loss应逐渐下降，而Accuracy则逐步提高。

这里TRAIN_EPOCH设置为20，IS_FIRST_TRAIN为True表示当前训练轮次为0，此时会从零开始训练，如果IS_FIRST_TRAIN为False则会导入模型参数，在之前训练过的模型的基础上继续训练。每一轮训练结束以后会保存一次模型。


In [9]:
# 本次训练的 epoch 数
TRAIN_EPOCH = 20

# 是否是第一次训练
IS_FIRST_TRAIN = True


def main():
    model = AlexNet(num_classes=10, channel=3)

    # 使用交叉熵作为损失函数
    loss_fn = nn.CrossEntropyLoss()

    # 使用随机梯度下降作为优化器，学习率设置为 0.01
    optimizer = nn.SGD(model.trainable_params(), 1e-2)

    # 如果不是第一次训练，那么导入之前保存的模型和之前累积训练的 epoch 数量
    if not IS_FIRST_TRAIN:
        param_dict = mindspore.load_checkpoint("model.ckpt")
        prev_epoch = param_dict['epoch']
        
        # 导入权重
        mindspore.load_param_into_net(model, param_dict)
    else:
        prev_epoch = 0

    # 开始训练
    for t in range(TRAIN_EPOCH):
        cur_epoch = t + prev_epoch + 1
        print(f"Epoch {cur_epoch}\n-------------------------------")

        model_train(model, train_dataset, loss_fn, optimizer)

        # 每次训练完一个 epoch 保存模型
        mindspore.save_checkpoint(model, "model.ckpt", append_dict={'epoch': cur_epoch})
        model_test(model, test_dataset, loss_fn)

训练模型

In [ ]:
main()

[WARNING] ME(226634:140282554931008,MainProcess):2025-03-28-17:56:36.513.000 [mindspore/nn/layer/basic.py:176] For Dropout, this parameter `keep_prob` will be deprecated, please use `p` instead.


Epoch 1
-------------------------------
loss: 2.304216  [  0/391]
loss: 2.302114  [100/391]
loss: 2.302338  [200/391]
loss: 2.302460  [300/391]
Test: 
 Accuracy: 10.0%, Avg loss: 2.302620 

Epoch 2
-------------------------------
loss: 2.302340  [  0/391]
loss: 2.302163  [100/391]
loss: 2.302531  [200/391]
loss: 2.302202  [300/391]
Test: 
 Accuracy: 10.0%, Avg loss: 2.302576 

Epoch 3
-------------------------------
loss: 2.303273  [  0/391]
loss: 2.303527  [100/391]
loss: 2.302502  [200/391]
loss: 2.301282  [300/391]
Test: 
 Accuracy: 10.0%, Avg loss: 2.302588 

Epoch 4
-------------------------------
loss: 2.302807  [  0/391]
loss: 2.302426  [100/391]
loss: 2.302949  [200/391]
loss: 2.302301  [300/391]
Test: 
 Accuracy: 10.0%, Avg loss: 2.302592 

Epoch 5
-------------------------------
loss: 2.303339  [  0/391]
loss: 2.303160  [100/391]
loss: 2.303053  [200/391]
loss: 2.302971  [300/391]
Test: 
 Accuracy: 10.0%, Avg loss: 2.302578 

Epoch 6
-------------------------------
loss: 2.30

#### **2.6 模型预测**

加载后的模型可以直接用于预测推理。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import mindspore

def predict(model, test_set):
    """预测
    Args:
        model   : 模型 
        test_set: 待预测的样本
    """
    model.set_train(False)
    for data, label in test_set.create_tuple_iterator():
        pred = model(data)
        predicted = pred.argmax(1)
        
        # 可视化预测结果
        plt.figure(figsize=(10, 5))
        for i in range(10):
            plt.subplot(2, 5, i + 1)
            # 反归一化并转换为图像格式
            image = data[i].asnumpy().transpose(1, 2, 0)
            image = np.clip(image, 0, 1)
            plt.imshow(image)
            plt.title(f'P:{predicted[i].item()}/A:{label[i].item()}')
            plt.axis('off')
        plt.tight_layout()
        plt.show()
        break

# 加载模型
model = AlexNet(num_classes=10, channel=3)
param_dict = mindspore.load_checkpoint("model.ckpt")
mindspore.load_param_into_net(model, param_dict)

# 进行预测并展示图片
predict(model, test_dataset)

NameError: name 'AlexNet' is not defined

### **3 总结**

本案例基于MindSpore框架，使用CASIA-WebFace数据集进行训练，LFW数据集进行测试，完成了人脸识别任务。我们对AlexNet模型进行了调整和优化，包括通道数的合并和新增卷积层与全连接层，以适应单GPU训练环境并提升模型性能。通过训练和评估，模型在测试集上表现良好，最终实现了准确的人脸识别预测。这一过程加深了对AlexNet模型结构和特性的理解，同时掌握了MindSpore框架的使用方法，为未来实际应用提供了支持。